In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from dist_s1_enumerator.dist_enum import enumerate_one_dist_s1_product
from dist_s1_enumerator.asf import get_rtc_s1_ts_metadata_from_mgrs_tiles
from shapely.geometry import Point
from datetime import datetime, timedelta
import geopandas as gpd
from rasterio.crs import CRS
import matplotlib.pyplot as plt
import contextily as ctx
import pandas as pd
from pathlib import Path

# Provide AOI

In [3]:
# Over Los Padres National Forest, CA
point = Point(-120.0048, 34.8923)
df_aoi = gpd.GeoDataFrame(geometry=[point], crs=CRS.from_epsg(4326))

# Get Overlapping Tiles

Can be point or polygon.

In [4]:
df_mgrs_tiles = get_mgrs_tiles_overlapping_geometry(point)

NameError: name 'get_mgrs_tiles_overlapping_geometry' is not defined

## Visualize Overlapping Tiles and AOI

In [ ]:
fig, ax = plt.subplots()
df_mgrs_tiles_plot = df_mgrs_tiles.copy()
df_mgrs_tiles_plot.geometry = df_mgrs_tiles_plot.geometry.boundary
df_mgrs_tiles_plot.plot(column='mgrs_tile_id', categorical=True, legend=True, ax=ax)
df_aoi.plot(ax=ax)
ctx.add_basemap(ax,
                crs=df_mgrs_tiles.crs.to_string(),
                source=ctx.providers.CartoDB.Voyager
                )

# Visualize Overlapping Passes

In [ ]:
MGRS_TILE = '10SGD'

In [ ]:
df_mgrs_burst_luts = get_lut_by_mgrs_tile_ids([MGRS_TILE])
df_mgrs_burst_luts.head()

In [ ]:
df_bursts = get_burst_table_from_mgrs_tiles([MGRS_TILE])
df_bursts.head()

In [ ]:
fig, ax = plt.subplots()
df_mgrs_tiles_plot = df_mgrs_tiles.copy()
df_mgrs_tiles_plot.plot(column='mgrs_tile_id', categorical=True, ax=ax, alpha=.2)
df_aoi.plot(ax=ax)
df_bursts_plot = df_bursts.copy()
df_bursts_plot.geometry = df_bursts_plot.geometry.boundary
df_bursts_plot.plot(column='track_number', categorical=True, ax=ax, legend=True)
leg = ax.get_legend()
leg.set_title('Track Numbers')
ctx.add_basemap(ax,
                crs=df_mgrs_tiles.crs.to_string(),
                source=ctx.providers.CartoDB.Voyager
                )

## Visualize specific group

In [ ]:
TRACK_NUMBER = 137

fig, ax = plt.subplots()
df_mgrs_tiles_plot = df_mgrs_tiles.copy()
df_mgrs_tiles_plot.plot(column='mgrs_tile_id', categorical=True, ax=ax, alpha=.2)
df_aoi.plot(ax=ax)
df_bursts_plot = df_bursts.copy()
df_bursts_plot.geometry = df_bursts_plot.geometry.boundary
df_bursts_plot = df_bursts_plot[df_bursts_plot.track_number == TRACK_NUMBER].reset_index(drop=True)
df_bursts_plot.plot(column='track_number', categorical=True, ax=ax, legend=True)
leg = ax.get_legend()
leg.set_title('Track Numbers')
ctx.add_basemap(ax,
                crs=df_mgrs_tiles.crs.to_string(),
                source=ctx.providers.CartoDB.Voyager
                )

# Get Candidate Post-Image set of RTC-S1 Data (Post-images)

Inspect possible set of post-images in the last 30 days.

In [ ]:
today = datetime.today()
df_rtc_post_possible = get_rtc_s1_ts_metadata_from_mgrs_tiles([MGRS_TILE], 
                                                     track_numbers=[TRACK_NUMBER],
                                                     start_acq_dt= today, 
                                                     stop_acq_dt= today - timedelta(days=30))
df_rtc_post_possible.head()

In [ ]:

fig, ax = plt.subplots()
df_mgrs_tiles_plot = df_mgrs_tiles.copy()
df_mgrs_tiles_plot.plot(column='mgrs_tile_id', categorical=True, ax=ax, alpha=.2)
df_aoi.plot(ax=ax)
df_rtc_plot = df_rtc_post_possible.copy()
df_rtc_plot.geometry = df_rtc_plot.geometry.boundary
df_rtc_plot.plot(column='acq_date_for_mgrs_pass', categorical=True, ax=ax, legend=True)
ctx.add_basemap(ax,
                crs=df_mgrs_tiles.crs.to_string(),
                source=ctx.providers.CartoDB.Voyager
                )

## Few minor notes
1. The geometries from our burst table differ from those collected from the DAAC (note the overlap)
2. In some cases, the burst ids collected from the latest pass will equal the total fixed bursts requests. In some cases, the actual data from the latest pass will be *less* than the requested bursts. This is due to S1 availability.

# Get inputs for 1 product

In [ ]:
post_date

In [ ]:
# most recent pass
post_date = df_rtc_post_possible.acq_dt.dt.date.max()
df_product = enumerate_one_dist_s1_product(MGRS_TILE, 
                                             track_number=TRACK_NUMBER,
                                             post_date=post_date,
                                             post_date_buffer_days=1
                                           )
df_product.head()

# Localizing Inputs

In [ ]:
from dist_s1_enumerator.rtc_s1_io import localize_rtc_s1_ts

In [ ]:
out_dir = Path('out')
out_dir.mkdir(exist_ok=True, parents=True)

In [ ]:
df_product_loc = localize_rtc_s1_ts(df_product, out_dir, max_workers=5)

In [ ]:
track_token = df_product_loc.track_token.iloc[0]
mgrs_tile_id = df_product_loc.mgrs_tile_id.iloc[0]
date_token = df_product_loc.acq_date_for_mgrs_pass.iloc[0]

df_product_loc.to_parquet(f'{mgrs_tile_id}__{track_token}__{date_token}_dist_s1_inputs.parquet')